In [ ]:
#References:
# https://github.com/npinto/opencv/blob/master/samples/python2/lk_track.py
# https://docs.opencv.org/4.x/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323
# https://amroamroamro.github.io/mexopencv/matlab/cv.goodFeaturesToTrack.html
# https://amroamroamro.github.io/mexopencv/matlab/cv.calcOpticalFlowPyrLK.html
# https://github.com/niconielsen32/ComputerVision/blob/master/opticalFlow/sparseOpticalFlow.py#L22

In [1]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt
import time

In [2]:
lk_params = dict(winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [3]:
feature_params = dict(maxCorners = 20,
                    qualityLevel = 0.3,
                    minDistance = 10,
                    blockSize = 7 )

In [4]:
trajectory_len = 40
detect_interval = 1
trajectories = []
frame_idx = 0

In [5]:
#live video capture
# cap = cv2.VideoCapture(0)

#using test video
cap = cv2.VideoCapture('test_video2.mp4')

In [6]:
suc, frame = cap.read()

#region of interest
r = cv2.selectROI("select the image",frame)
x_roi,y_roi,width,height = r[0],r[1],r[2],r[3]
print(x_roi,y_roi,width,height)

#Press 0 to exit from selection window
cv2.waitKey(0)
cv2.destroyAllWindows()

648 355 350 321


In [7]:
while True:


    suc, frame = cap.read()
    
    #region of interest
    roi = frame[int(y_roi):int(y_roi+height),int(x_roi):int(x_roi+width)]
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)


    # Calculate optical flow for a sparse feature set using the iterative Lucas-Kanade Method
    # https://docs.opencv.org/4.x/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323
    if len(trajectories) > 0:
        img0, img1 = prev_gray, roi_gray
        p0 = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2)

        p1, _st, _err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)

        p0r, _st, _err = cv2.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)

        d = abs(p0-p0r).reshape(-1, 2).max(-1)
        good = d < 1

        new_trajectories = []

         # Get all the trajectories
        for trajectory, (x, y), good_flag in zip(trajectories, p1.reshape(-1, 2), good):
            if not good_flag:
                continue
            trajectory.append((x, y))
            if len(trajectory) > trajectory_len:
                del trajectory[0]
            new_trajectories.append(trajectory)
            # Newest detected point
            cv2.circle(roi, (int(x), int(y)), 2, (0, 0, 255), -1)

        trajectories = new_trajectories

        # Draw all the trajectories
        cv2.polylines(roi, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0))
        cv2.putText(roi, 'track count: %d' % len(trajectories), (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)


     # Update interval - When to update and detect new features
    if frame_idx % detect_interval == 0:

        mask = np.zeros_like(roi_gray)
        mask[:] = 255

         # Lastest point in latest trajectory
        for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:
            cv2.circle(mask, (x, y), 5, 0, -1)

        # selecting corners from ROI
        p = cv2.goodFeaturesToTrack(roi_gray, mask = mask, **feature_params)

        #adding corners to trajectory
        if p is not None:
            for x, y in np.float32(p).reshape(-1, 2):
                trajectories.append([(x, y)])


    frame_idx += 1
    prev_gray = roi_gray

    cv2.imshow('Optical Flow', roi)
    cv2.imshow('Mask', mask)
    
    #Press q to exit from selection window
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()